In [15]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests

In [16]:
df = pd.read_excel('175Pauls.xlsx')
df = df.apply(lambda x: x.str.strip())
df.head()

,Signatur
0,"RB 0, Katalog - 1"
1,"RB 0, Katalog - 2"
2,"RB 0, Katalog - 3"
3,"RB 0, Katalog - 4"
4,"RB 0, Katalog - 5"


In [17]:
base_url = 'https://services.dnb.de/sru/dnb?'
params = {'recordSchema' : 'MARC21plus-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
         }

In [18]:
def gnd_abfrage(row):
    params.update({'query': f'SIG = "{row.Signatur}*"'})
    response = requests.get(base_url, params=params)
    response_xml = soup(response.content, features="xml")
    

    try:
        matches = list()
        for record in response_xml.find_all('record', {'type':'Bibliographic'}):
            record_match = dict()
            #idn bestimmen
            try:
                record_match['idn'] = record.find('controlfield', {'tag': '001'}).string.strip()
            except:
                record_match['idn'] = None

            matches.append(record_match)
        return matches
    except Exception as e:
        return "fehler %r" % e

In [19]:
df['idntitel'] = df.apply(gnd_abfrage, axis=1)
df

FeatureNotFound: Couldn't find a tree builder with the features you requested: xml. Do you need to install a parser library?